# 3 SNARKs Prelude: Elliptic Curves and Polynomial Commitments

## 3.4 KZG Commitments

We will use the BLS12-381 curve as this is the most common curve used in KZG due 
to its security properties. However, note that KZG can work with other curves 
that are pairing-friendly (including BN254). Factors such as speed, security, 
and ease of implementation impact the choice of curves.

For more info, see https://docs.gnark.consensys.io/Concepts/schemes_curves

In [102]:
import random
import galois
import numpy as np
from py_ecc.bls12_381 import (
    is_on_curve,
    G1,
    G2,
    multiply,
    add,
    neg,
    b,
    b2,
    curve_order,
    pairing
)

In [103]:
def generate_s_points(generator, scalar, b, max_degree):
    """
    Generates a sequence of points by repeatedly multiplying a generator point with increasing powers of a scalar.
    
    Args:
        generator: The base point (either G1 or G2) to start the sequence
        scalar: The secret value to be used for multiplying with the generator
        b: The curve parameter (b for G1 or b2 for G2)
        max_degree: The maximum degree of the polynomial
    
    Returns:
        list: A list of points [g, g*s, g*s^2, ..., g*s^max_degree]
    """

    s_points = []
    for i in range(max_degree + 1):
        if i == 0:
            s_points.append(multiply(generator, int(scalar**i)))
        else:
            s_points.append(multiply(s_points[i - 1], int(scalar)))
        assert is_on_curve(s_points[i], b), "Point not on curve"
    return s_points

In [104]:
def setup(max_degree):
    """
    Initializes the trusted setup for KZG commitments by generating the finite 
    field and SRS (structured reference strings).
    
    Args:
        max_degree: The maximum polynomial degree supported by the setup
    
    Returns:
        tuple: (gf_q, g1_points, g2_points) where gf_q is the finite field of 
        order matching the curve order, and g1_points, g2_points are lists of points 
        representing powers of a secret value
    """

    # Create the finite field of order from BLS12-381
    # Note: this has moved out of setup() because it takes a longer time to run
    gf_q = galois.GF(curve_order)

    # Generate [s^0], [𝑠^1], …, [s^max_degree]
    # s_points = [[s^0=1], [s^1], [s^2], ..., [s^max_degree]]
    s = gf_q.Random()

    # BLS12-381 operates in 3 groups, G1, G2, and GT(target group)
    # We need to generate a set of points in G1 and G2 separately
    g1_points = generate_s_points(G1, s, b, max_degree)
    g2_points = generate_s_points(G2, s, b2, max_degree)

    return gf_q, g1_points, g2_points

# Set the maximum degree of the polynomial that we want to support
max_degree = 10

# Run the trusted setup
gf_q, g1_points, g2_points = setup(max_degree)

print("gf_q.properties: ", gf_q.properties)
print("g1_points: ", g1_points)
print("g2_points: ", g2_points)

gf_q.properties:  Galois Field:
  name: GF(52435875175126190479447740508185965837690552500527637822603658699938581184513)
  characteristic: 52435875175126190479447740508185965837690552500527637822603658699938581184513
  degree: 1
  order: 52435875175126190479447740508185965837690552500527637822603658699938581184513
  irreducible_poly: x + 52435875175126190479447740508185965837690552500527637822603658699938581184506
  is_primitive_poly: True
  primitive_element: 7
g1_points:  [(3685416753713387016781088315183077757961620795782546409894578378688607592378376318836054947676345821548104185464507, 1339506544944476473020471379941921221584933875938349620426543736416511423956333506472724655353366534992391756441569), (1969592380550290362689667904795098666626151120559509861646614897530129525382872868692923736706714859262152224363964, 628048017431530234275505501223530098434719704464593649075152200014451364779659245249750852134810330941936513501708), (31552547252561998060350681448071746529355175340

In [105]:
def points_to_polynomial(polynomial_points):
    """
    Converts a set of points to a polynomial using Lagrange interpolation.
    
    Args:
        polynomial_points: List of tuples (x, y) representing points the polynomial should pass through
    
    Returns:
        galois.Poly: The interpolated polynomial passing through all given points
    """

    # Extract x and y coordinates separately
    point_xs = [point[0] for point in polynomial_points]
    point_ys = [point[1] for point in polynomial_points]

    point_xs_np = np.array(point_xs, dtype=object).view(gf_q)
    point_ys_np = np.array(point_ys, dtype=object).view(gf_q)

    polynomial = galois.lagrange_poly(point_xs_np, point_ys_np)
    return polynomial

# Array of points that the polynomial pass through
polynomial_points = [(1, 10), (2, 100), (3, 20), (4, 43)]

polynomial = points_to_polynomial(polynomial_points)
print("polynomial: ", polynomial)
print("polynomial.coefficients: ", polynomial.coefficients())
print("polynomial.degree: ", polynomial.degree)

polynomial:  26217937587563095239723870254092982918845276250263818911301829349969290592302x^3 + 52435875175126190479447740508185965837690552500527637822603658699938581184155x^2 + 26217937587563095239723870254092982918845276250263818911301829349969290593102x + 52435875175126190479447740508185965837690552500527637822603658699938581183990
polynomial.coefficients:  [26217937587563095239723870254092982918845276250263818911301829349969290592302
 52435875175126190479447740508185965837690552500527637822603658699938581184155
 26217937587563095239723870254092982918845276250263818911301829349969290593102
 52435875175126190479447740508185965837690552500527637822603658699938581183990]
polynomial.degree:  3


In [106]:
def sumproduct_polynomial_spoints(polynomial, s_points):
    """
    Computes the sum of products between polynomial coefficients and points in the SRS.
    
    Args:
        polynomial: The polynomial
        s_points: The structured reference string points
    
    Returns:
        Point: Result of ∑(c_i * [s^i]) where c_i are polynomial coefficients
    """    
    polynomial_coefficients_asc = polynomial.coefficients(order="asc")
    result = None
    for i in range(polynomial.degree + 1):
        result = add(result, multiply(s_points[i], int(polynomial_coefficients_asc[i])))
    return result

In [107]:
def commit(polynomial, s_points):
    """
    Computes the commitment to a polynomial using the SRS.
    
    Args:
        polynomial: The polynomial
        s_points: The structured reference string points
    
    Returns:
        Point: The commitment to the polynomial
    """
    assert polynomial.degree <= len(
        s_points
    ), "Polynomial must have a degree less than or equal to the number of s_points"

    # Compute the commitment
    return sumproduct_polynomial_spoints(polynomial, s_points)

commitment = commit(polynomial, g1_points)
print("commitment: ", commitment)

commitment:  (2718506277867648741138682242890197239750968365831691044804779700704722507355493290652183013010431968799335451929631, 3085708656674622502844192737841937956562376080365213989486950579495059078041876500908403722935195764274412535801223)


In [108]:
def open(
    polynomial,
    opening_point,
    s_points,
):
    """
    Generates a proof of evaluation for specific points of the polynomial.
    
    Args:
        polynomial: The committed polynomial
        opening_point: List of (x, y) points to prove
        s_points: The structured reference string points
    
    Returns:
        Point: The opening proof [Q(s)]
    """

    # Calculate the Quotient Polynomial
    opening_point_polynomial = points_to_polynomial(opening_point)
    p_x_minus_y = polynomial - opening_point_polynomial

    # For each point in the opening_points, multiply denominator by (X - x_opening_point)
    # This builds the polynomial that has roots at all the x-coordinates
    x_minus_opening_point = galois.Poly([1], gf_q)
    for x_opening_point in opening_point:
        x_minus_opening_point = x_minus_opening_point * (galois.Poly([1, -x_opening_point[0]], gf_q))
    
    # P(X) - y / (X - z)
    quotient, reminder = divmod(p_x_minus_y, x_minus_opening_point)

    # Check the remainder is zero
    assert reminder == 0, "Remainder is not zero"

    # Calculate the opening proof from the Quotient Polynomial
    opening_proof = sumproduct_polynomial_spoints(quotient, s_points)
    return opening_proof

# Calculate opening proof, [Q(s)]
opening_points = [(1, 10), (3, 20)]
opening_proof = open(polynomial, opening_points, g1_points)
print("opening_proof: ", opening_proof)

opening_proof:  (1539944504990102013070273056638562273050484494659125507190773574144215825599149626270778694879242262198255072541627, 3756102706666338624108360584966874202039595911748923763056420484031478119157416514619813769481366826888970827341005)


In [109]:
def verify(
    opening_point,
    opening_proof,
    commitment,
):
    """
    Verifies a KZG proof of evaluation.
    
    Args:
        opening_point: List of (x, y) points being proven
        opening_proof: The proof point [Q(s)]
        commitment: The original commitment [P(s)]
    
    Returns:
        bool: True if the proof is valid, False otherwise
    """

    # Create polynomial z(X) = ∏(X - x_i)
    opening_point_xs = [point[0] for point in opening_point]
    z = galois.Poly([1], gf_q)
    for x in opening_point_xs:
        z = z * galois.Poly([gf_q(1), -x], gf_q)

    # Compute [z]
    z_armor = sumproduct_polynomial_spoints(z, g2_points)
    lhs = pairing(z_armor, opening_proof)
    
    # Calculate the right hand side of the equation
    opening_point_polynomial = points_to_polynomial(opening_point)
    y_armor = sumproduct_polynomial_spoints(opening_point_polynomial, g1_points)
    rhs = pairing(G2, add(commitment, neg(y_armor)))

    # verify pair([Q(s)], [s] − [z]) = pair([P(s)] − [y], [1]) 
    # i.e. verify pair(opening_proof, g2_points - point * G2) == e(commitment - [opening_point_polynomial], G2)
    return lhs == rhs

assert verify(opening_point, opening_proof, commitment), "Verification failed"